In [16]:
import numpy as np 
import pandas as pd
import re

In [17]:
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


In [18]:
df = pd.read_csv(r'A:\Work Docs\Data Analyst work\Campus X\10 Streamlit\Capstone\gurgaon_properties_cleaned_v1.csv')

In [19]:
df.head(1)

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,flat,signature global park 4,sector 36,0.82,7585.0,1081.0,Super Built up area 1081(100.43 sq.m.)Carpet area: 650 sq.ft. (60.39 sq.m.),3,2,2,not available,2.0,NaN,0 to 1 Year Old,"['Sector 55-56 Metro Station', 'The Leaf Mall Sohna', 'Omaxe Gurgaon Mall', 'Badshahpur Sohna Rd Hwy', 'Vidya Niketan School', 'KIIT College of Engg', 'Scottish High International School', 'Rawal Institutions', 'Park Hospital', 'Indira Gandhi Intl Airport', 'Spaze Business Park', 'DLF Cyber City']",NaN,"['Lift(s)', 'Park']"


Focus here is on columns->
areaWithType,
additionaRoom,
agePossesion,
furnishedDetails,
features

In [20]:
df[['price', 'area','areaWithType']].sample(5)


,price,area,areaWithType
1364,2.65,3150.0,Super Built up area 3150(292.64 sq.m.)
3360,1.20,1760.0,Super Built up area 1760(163.51 sq.m.)Built Up area: 1186 sq.ft. (110.18 sq.m.)Carpet area: 1130 sq.ft. (104.98 sq.m.)
2509,0.48,1250.0,Super Built up area 1250(116.13 sq.m.)Carpet area: 770 sq.ft. (71.54 sq.m.)
1465,2.04,1450.0,Super Built up area 1852(172.06 sq.m.)Carpet area: 1450 sq.ft. (134.71 sq.m.)
2873,2.55,2150.0,Super Built up area 2150(199.74 sq.m.)Built Up area: 2000 sq.ft. (185.81 sq.m.)


making three new columns with areaWithType--> CarpetArea, buitlupArea, SuperBuiltupArea

In [21]:
#this function extracts  the Super Builtup area
def get_super_built_up_area(text):
    match = re.search(r'Super Built up area (\d+\.?\d*)',text)
    if match:
        return float(match.group(1))
    return None


In [22]:
#this function extracts  the  Builtup area or Carpet area
def get_area(text, area_type):
    match = re.search(area_type + r'\s*:\s*(\d+\.?\d*)',text)
    if match:
        return float(match.group(1))
    return None

In [23]:
# This function checks if the area is provided in sq.m. and converts it to sqft if needed
def convert_to_sqft(text, area_value):
    if area_value is None:
        return None
    match = re.search(r'{} \((\d+\.?\d*) sq.m.\)'.format(area_value), text)
    if match:
        sq_m_value = float(match.group(1))
        return sq_m_value * 10.7639  # conversion factor from sq.m. to sqft
    return area_value

In [24]:
# Extract Super Built up area and convert to sqft if needed
df['super_built_up_area'] = df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['super_built_up_area']), axis=1)

# Extract Built Up area and convert to sqft if needed
df['built_up_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Built Up area'))
df['built_up_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['built_up_area']), axis=1)

# Extract Carpet area and convert to sqft if needed
df['carpet_area'] = df['areaWithType'].apply(lambda x: get_area(x, 'Carpet area'))
df['carpet_area'] = df.apply(lambda x: convert_to_sqft(x['areaWithType'], x['carpet_area']), axis=1)



In [25]:
df[['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)


,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
2821,1.15,flat,1660.0,Super Built up area 1660(154.22 sq.m.)Carpet area: 1282 sq.ft. (119.1 sq.m.),1660.0,NaN,1282.0
3320,0.45,flat,598.0,Carpet area: 598 (55.56 sq.m.),NaN,NaN,598.0
2675,1.90,flat,1975.0,Built Up area: 1975 (183.48 sq.m.),NaN,1975.0,NaN
3561,0.99,flat,1205.0,Super Built up area 1205(111.95 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),1205.0,NaN,900.0
1231,5.00,house,4000.0,Built Up area: 4000 (371.61 sq.m.),NaN,4000.0,NaN


In [26]:
df.duplicated().sum()

122